In [ ]:
import h5py
import pickle
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
import random
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras import layers,models,optimizers,callbacks,constraints
from tensorflow.keras.constraints import Constraint
from tensorflow.keras.layers import Input, LSTM,GRU,TimeDistributed,Dropout, RepeatVector, Dense, Conv2D,multiply,Lambda,Add,Concatenate, Multiply,Conv2DTranspose,Layer, Reshape, ZeroPadding2D,Flatten, MaxPool3D, UpSampling2D, BatchNormalization,LeakyReLU
from tensorflow.keras.models import Sequential, Model 
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping,Callback,LearningRateScheduler,ModelCheckpoint
from sklearn.utils import shuffle
import math
import os
from tensorflow.keras import regularizers
from scipy.linalg import qr
tf.random.set_seed(46)
gpus = tf.config.list_physical_devices('GPU')
W=20
hdf5_file ='data/moreall_w20.hdf5'
hdf5 = h5py.File(hdf5_file, 'r')
N=0
hid_dim=5

number_list = list(np.arange(15))
train_idx=list(np.arange(45))
test_idx=[]

for i in range (3):
    w=i*15
    test_idx.append(np.array(random.sample(number_list,3))+w)
    
test_idx=np.array(test_idx).reshape(-1,1)

# for selected_value in test_idx:
#     train_idx.remove(selected_value)
# test_idx=list(test_idx.reshape(1,-1))

# np.save('train_label',train_idx)
# np.save('test_label',test_idx[0])

h5k=[list(hdf5.keys())[i] for i in train_idx]

for V in h5k:
    t=len(list(hdf5[V].keys())[:-1])
    N+=t
    
idx={i:[] for i in range (N)}

x=0



for V in h5k:
    
    n =list(hdf5[V].keys())[:-1]
    for fn in n:
        idx[x].append(V)
        idx[x].append(fn)
        
#         print(idx[x])
        x+=1

index=shuffle(idx)
# print(index)

def data_generator(idx, hdf5,batch_size,if_train = True):
    i = 0

    while True:
        X = []
        Y = []
        for b in range(batch_size):
            if i == len(idx):
                i = 0
            alll=idx[i]
            x = np.expand_dims(np.array(hdf5[alll[0]][alll[1]],dtype=np.float64),axis=-1)/255  # read dataset on the fly
#             x=np.rollaxis(x,1,0)
            x=np.nan_to_num(x)
            
            y_idx='trial_%04i' % int(int(alll[1][-4:])+1)
            y = np.expand_dims(np.array(hdf5[alll[0]][y_idx],dtype=np.float64),axis=-1)/255  # read dataset on the fly
#             y=np.rollaxis(y,1,0)
            y=np.nan_to_num(y)
            
            
            X.append(x)
            Y.append(y)
            i += 1

        X = np.asarray(X)
        Y = np.asarray(Y)
        
        
        yield [X,Y],[X,Y]

        
## input layer
class MSE_UNSUP(Layer):

    """ Identity transform layer that adds KL divergence
    to the final model loss.
    """

    def __init__(self, *args, **kwargs):
        self.is_placeholder = True
        super(MSE_UNSUP, self).__init__(*args, **kwargs)
    def call(self,inputs):
        D,A=inputs

        L=tf.keras.losses.mse(D,A)
        L=tf.reduce_mean(L)
        self.add_loss(L*20*128*128, inputs=inputs)
        
        return inputs,L
class KLDivergenceLayer(Layer):

    """ Identity transform layer that adds KL divergence
    to the final model loss.
    """

    def __init__(self, *args, **kwargs):
        self.is_placeholder = True
        super(KLDivergenceLayer, self).__init__(*args, **kwargs)

    def call(self, inputs):

        mu, log_var = inputs

        kl_batch =  -0.5 * K.sum(1 + log_var -
                                K.square(mu) - K.exp(log_var), axis=-1)

        self.add_loss(tf.reduce_mean(kl_batch), inputs=inputs)

        return inputs,tf.reduce_mean(kl_batch)

input0 = Input((W,128,128, 1))
input1 = Input((W,128,128, 1))


def encoder():
    model = Sequential()
    model.add(TimeDistributed(Conv2D(16, (5,5), strides= (2,2),padding='same', kernel_initializer="he_normal"),input_shape = (W,128,128, 1)))
    model.add(LeakyReLU(alpha=0.05))
    model.add(BatchNormalization())
    model.add(TimeDistributed(Conv2D(16, (5,5), strides= (2,2),padding='same', kernel_initializer="he_normal")))
    model.add(LeakyReLU(alpha=0.05))
    model.add(BatchNormalization())
    model.add(TimeDistributed(Conv2D(32, (5,5), strides= (2,2),padding='same', kernel_initializer="he_normal")))
    model.add(LeakyReLU(alpha=0.05))
    model.add(BatchNormalization())
    model.add(TimeDistributed(Conv2D(32, (5,5), strides= (2,2),padding='same', kernel_initializer="he_normal")))
    model.add(LeakyReLU(alpha=0.05))
    model.add(BatchNormalization())
    model.add(TimeDistributed(Conv2D(64, (5,5), strides= (2,2),padding='same', kernel_initializer="he_normal")))
    model.add(LeakyReLU(alpha=0.05))
    model.add(BatchNormalization())

    model.add(TimeDistributed(Flatten()))                           
#     model.summary()
    return model
encoder()
hidden=encoder()(input0)
encode=GRU(32)(hidden)                     

z_mu = Dense(hid_dim,kernel_regularizer='l1_l2')(encode)
z_log_var = Dense(hid_dim,kernel_regularizer='l1_l2')(encode)

[A, z_log_var_A2],loss3 = KLDivergenceLayer()([z_mu, z_log_var])
z_sigma_A = Lambda(lambda t: K.exp(.5*t))(z_log_var_A2)

eps_A = K.random_normal(shape=(K.shape(input0)[0],hid_dim))
z_eps_A2 = Multiply()([z_sigma_A, eps_A])#Multiply()([z_sigma_A, eps_A])
z_A = Add()([A, z_eps_A2])

decode = RepeatVector(W)(z_A)
decode = GRU(32, return_sequences=True)(decode)
decode = GRU(hidden.shape[-1], return_sequences=True)(decode)

decode_future = RepeatVector(W)(z_A)
decode_future = GRU(32, return_sequences=True)(decode_future)
decode_future = GRU(hidden.shape[-1], return_sequences=True)(decode_future)

def decoder1():
    model1 = Sequential()
    model1.add(TimeDistributed(Reshape((4,4,64))))
    model1.add(TimeDistributed(Conv2DTranspose(32, (5,5), strides= (2,2),padding='same', kernel_initializer="he_normal")))
    model1.add(LeakyReLU(alpha=0.05))
    model1.add(BatchNormalization())
    model1.add(TimeDistributed(Conv2DTranspose(32, (5,5), strides= (2,2),padding='same', kernel_initializer="he_normal")))
    model1.add(LeakyReLU(alpha=0.05))
    model1.add(BatchNormalization())
    model1.add(TimeDistributed(Conv2DTranspose(16, (5,5), strides= (2,2),padding='same', kernel_initializer="he_normal")))
    model1.add(LeakyReLU(alpha=0.05))
    model1.add(BatchNormalization())

    model1.add(TimeDistributed(Conv2DTranspose(16, (5,5), strides= (2,2),padding='same', kernel_initializer="he_normal")))
    model1.add(LeakyReLU(alpha=0.05))
    model1.add(BatchNormalization())

    
    model1.add(TimeDistributed(Conv2DTranspose(1, (5,5), strides= (2,2),activation='sigmoid',padding='same', kernel_initializer="he_normal")))

    return model1

def decoder2():
    model1 = Sequential()
    model1.add(TimeDistributed(Reshape((4,4,64))))
    model1.add(TimeDistributed(Conv2DTranspose(32, (5,5), strides= (2,2),padding='same', kernel_initializer="he_normal")))
    model1.add(LeakyReLU(alpha=0.05))
    model1.add(BatchNormalization())
    model1.add(TimeDistributed(Conv2DTranspose(32, (5,5), strides= (2,2),padding='same', kernel_initializer="he_normal")))
    model1.add(LeakyReLU(alpha=0.05))
    model1.add(BatchNormalization())
    model1.add(TimeDistributed(Conv2DTranspose(16, (5,5), strides= (2,2),padding='same', kernel_initializer="he_normal")))
    model1.add(LeakyReLU(alpha=0.05))
    model1.add(BatchNormalization())

    model1.add(TimeDistributed(Conv2DTranspose(16, (5,5), strides= (2,2),padding='same', kernel_initializer="he_normal")))
    model1.add(LeakyReLU(alpha=0.05))
    model1.add(BatchNormalization())

    
    model1.add(TimeDistributed(Conv2DTranspose(1, (5,5), strides= (2,2),activation='sigmoid',padding='same', kernel_initializer="he_normal")))

    return model1
out1=decoder1()(decode)
out2=decoder2()(decode_future)

[out1,_],loss1=MSE_UNSUP()([out1,input0])
[out2,_],loss2=MSE_UNSUP()([out2,input1])



allmodel = Model(inputs=[input0,input1], outputs=[out1,out2])

allmodel.add_metric(loss3, "KL_loss")
allmodel.add_metric(loss2, "future_loss")

allmodel.add_metric(loss1, "current_loss")

allmodel.summary()

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001,clipvalue=1.0)
allmodel.compile( optimizer=optimizer,loss='mse')
bs=64

# history1 =allmodel.fit( data_generator(index,hdf5,bs,True),validation_data=data_generator(index[-1000:],hdf5, 10,False),
#                        batch_size=bs, epochs=100,validation_steps=np.ceil(1000/2-1),verbose=1, steps_per_epoch=np.ceil((len(idx))/300-1))
allmodel.load_weights("test_model_morelatent5_20.h5")
# allmodel.save_weights("split3_latent5_20.h5")